In [1]:
import math
import numpy as np
import cv2
import mediapipe as mp
from PIL import Image
import os
from tqdm import tqdm

In [2]:
def transform(path):
    '''Загружает файл с указанного пути. 
    Возвращает вырезанное изображение руки (32,32) и флаг что изображение найдено. '''
    # считываем файл
    im = cv2.imread(path) 
    flag = False
    resized_image = []
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) 
    # Находим контуры изображений
    ret, thresh = cv2.threshold(imgray, 50, 255, 0) 
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
    perimetr = []
    # Определяем что контуры найдены
    if contours:
        # Устаналвливаем флаг в значение "удачно"
        flag = True
        # Ищем длинну контуров и находим контур с самой большой длинной
        for c in contours:
            perimetr.append(cv2.arcLength(c, True))
        max_ = np.argmax(perimetr)
        # ищем баунтин бокс самого большого контура, и вырезаем его из изображения
        x,y,w,h =cv2.boundingRect(contours[max_])
        img = im[y:y+h, x:x+w]
        # задаём размеры изображения.
        resized_image = cv2.resize(img, (32, 32))
    return resized_image, flag    
    


In [3]:
def verify(path):
    '''Проверяет на корректность полученного файла изображения'''
    try:
        im = Image.open(path)
        im.verify() 
        im.close()
        im = Image.open(path) 
        # быстрое преобразование, покажет есть ли изьян в данных
        im.transpose(Image.FLIP_LEFT_RIGHT)
        im.close()
    except: 
        print(f'Ошибка верификации файла: {path}')
        return 1
    return 0

In [4]:
def preprocessing():
    '''функция обработки данных'''
    ver = 0
    err_detect = 0
    i = 0
    # список который папок изображения которых будут использоваться в качестве валидации
    list_test = ['00', '09']
    classes = {'01_palm':1, '02_l':2, '03_fist':3, '04_fist_moved':4, '05_thumb':5, '06_index':6, '07_ok':7, 
               '08_palm_moved':8, '09_c':9, '10_down':10}
    os.mkdir('C:\\train')
    os.mkdir('C:\\test')
    
    for cl in classes.keys():
        os.mkdir('C:\\train\\'+cl)
        os.mkdir('C:\\test\\'+cl)

    for d in tqdm(os.listdir('C:\\leapGestRecog')):
        for cl in classes.keys():
            path = 'C:\\leapGestRecog\\'+ d + '\\'+cl
            for filename in os.listdir(path):
                i += 1
                try:                    
                    img_, flag = transform(path +'\\' + filename)
                    
                    if flag:
                        if d in list_test:
                            path_save = 'C:\\test\\'+cl+'\\'+filename
                            cv2.imwrite(path_save, img_)
                            ver += verify(path_save)
                        else:
                            path_save = 'C:\\train\\'+cl+'\\'+filename                 
                            cv2.imwrite(path_save, img_)   
                            ver += verify(path_save)                            
                except:
                    print (f"Ошибка записи - {filename}")
                    err_detect += 1
    print(f'Файлов обработанно - {i}')     
    print(f'Ошибок записи - {err_detect}')
    print(f'Ошибок верификации - {ver}')               
    print(f'Ошибок записи - {err_detect}')                

In [5]:
preprocessing()

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [04:37<00:00, 27.71s/it]

Файлов обработанно - 20000
Ошибок записи - 0
Ошибок верификации - 0
Ошибок записи - 0
